In [ ]:
# Load Parquet file
import pandas as pd

# Path to the Parquet file
path = "/Users/path"

# Read the Parquet file
df = pd.read_parquet(path)


In [ ]:
import openai
from openai import OpenAI
import pandas as pd
from tqdm.auto import tqdm
import concurrent.futures
import threading
import time
import random
import os

# Register tqdm with pandas
tqdm.pandas()

# ====== Put your API keys here ======
api_key_list = []
# ===================================

# Thread-local storage (each thread keeps its own API key / client)
thread_local = threading.local()

def get_client():
    """Get an OpenAI client with a per-thread rotated API key."""
    if not hasattr(thread_local, "client"):
        thread_id = threading.get_ident()
        api_key = api_key_list[thread_id % len(api_key_list)]
        thread_local.client = OpenAI(api_key=api_key)
    return thread_local.client


# ===================== ① LLM call =====================

def classify_author_region_country(author_str, year=None, max_retries=3):
    """
    Infer (Region, Country) from an author string and an optional publication year.

    The LLM output MUST be exactly one line:
      Region: <region>, Country: <country>
    """

    # Skip if Author is missing
    if pd.isna(author_str) or str(author_str).strip() == "":
        return "Skipped: Author missing"

    author_str = str(author_str).strip()

    # Handle Year
    if year is None or pd.isna(year) or str(year).strip() == "":
        year_str_for_prompt = "unknown"
        year_info_text = (
            "The publication year is unknown. Ignore year information and rely solely on "
            "surname origin and typical geographic usage."
        )
    else:
        try:
            year_int = int(float(year))
            year_str_for_prompt = str(year_int)
        except Exception:
            year_str_for_prompt = str(year).strip()

        year_info_text = (
            f"The publication year is {year_str_for_prompt}. "
            "This can be used as a very weak hint only, but the surname origin must remain "
            "the primary basis for determining region/country."
        )

    # ==== Prompt (force single-line output) ====
    prompt = f"""
You are an expert in global surname origins and geography.
Given an author string and a publication year, infer the most likely macro-region and modern country
of the FIRST author.

Follow these rules strictly:
1. Classify primarily based on the origin and typical geographic distribution of the first surname.
2. Use the publication year only as a very weak secondary hint.
3. If the year is unknown, ignore it.
4. If the classification is genuinely uncertain, output 'Unknown'.
5. Region must be exactly one of:
   Europe / Middle East / Asia / Africa / North America / Latin America / Unknown
6. Country must be a modern country name in English. If uncertain → 'Unknown'.

IMPORTANT OUTPUT CONSTRAINTS:
- You MUST output exactly ONE line.
- The line MUST start with 'Region:'.
- The line MUST contain a comma after the region part.
- You MUST NOT output any explanations, analysis, or reasoning.
- DO NOT output words like 'because', 'Chain-of-thought', or any justification.

Output format (MUST follow this exactly):
Region: <one of the 6 regions or Unknown>, Country: <country name or Unknown>

Examples (OUTPUT FORMAT ONLY):

Example 1
Input:
  Author string: "Müller and Zimmer, 1936"
  Publication year: 1936
Output:
  Region: Europe, Country: Germany

Example 2
Input:
  Author string: "Nguyen, 1995"
  Publication year: 1995
Output:
  Region: Asia, Country: Vietnam

Now classify the following record.

Author string: "{author_str}"
Publication year: {year_str_for_prompt}

{year_info_text}

Remember: output ONLY one line in this format:
Region: <region>, Country: <country>
"""

    client = get_client()

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4.1-mini",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=20,  # Keep small to encourage a single-line output
                temperature=0
            )
            return response.choices[0].message.content.strip()

        except Exception as e:
            if attempt < max_retries - 1:
                # Retry on failures such as rate limits
                wait = (2 ** attempt) + random.uniform(0, 1)
                time.sleep(wait)
            else:
                return f"Error after {max_retries} attempts: {e}"

    return "Error: Max retries exceeded"


# ===================== ② Parsing =====================

def parse_region_country(llm_str):
    """
    Parse the LLM output string, e.g.:
      'Region: Europe, Country: Germany'
    into a dict.
    """
    result = {"Region": "Unknown", "Country": "Unknown"}

    if not isinstance(llm_str, str):
        return result
    if "Skipped" in llm_str or "Error" in llm_str:
        return result

    try:
        parts = [p.strip() for p in llm_str.split(",")]
        for part in parts:
            if ":" not in part:
                continue
            key, value = part.split(":", 1)
            key = key.strip().lower()
            value = value.strip()
            if key == "region":
                result["Region"] = value
            elif key == "country":
                result["Country"] = value
    except Exception:
        pass

    return result


# ===================== ③ Parallel processing =====================

def process_single_item(args):
    """Process a single row (for parallel execution)."""
    idx, author_str, year = args

    if pd.isna(author_str) or str(author_str).strip() == "":
        return idx, "Skipped: Author missing"

    try:
        result = classify_author_region_country(author_str, year)
        return idx, result
    except Exception as e:
        return idx, f"Error: {e}"


def process_batch_parallel(batch_data, batch_id, max_workers=15):
    """Process one batch in parallel."""
    print(f"\nStarting batch {batch_id} ({len(batch_data)} records)")

    # Target Author / Year columns in df
    process_args = [
        (idx, row["Author"], row["Year"])
        for idx, row in batch_data.iterrows()
    ]

    results = {}

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_idx = {
            executor.submit(process_single_item, args): args[0]
            for args in process_args
        }

        for future in tqdm(
            concurrent.futures.as_completed(future_to_idx),
            total=len(future_to_idx),
            desc=f"Batch {batch_id}"
        ):
            idx = future_to_idx[future]
            try:
                _, result = future.result()
                results[idx] = result
            except Exception as e:
                print(f"Error occurred (idx {idx}): {e}")
                results[idx] = f"Error: {e}"

    ordered_results = [results[idx] for idx in batch_data.index]
    print(f"Finished batch {batch_id}")
    return ordered_results


# ===================== ④ Main processing =====================

def main_processing():
    """
    Main pipeline:
    Assign (Region, Country) to df based on Author / Year columns.
    """
    global df  # Assume df exists globally

    print("=" * 60)
    print("Starting LLM-based author region/country classification")
    print("=" * 60)

    print(f"Total records: {len(df):,}")

    # Validate required columns
    missing_cols = [c for c in ["Author", "Year"] if c not in df.columns]
    if missing_cols:
        print(f"❌ Error: Missing required columns: {missing_cols}")
        print(f"Available columns: {df.columns.tolist()}")
        return

    # Batch settings
    batch_size = 3000
    max_workers = 25
    total_batches = (len(df) + batch_size - 1) // batch_size

    print(f"Batch size: {batch_size}")
    print(f"Total batches: {total_batches}")
    print(f"Parallel workers: {max_workers}")

    # Output directory
    save_dir = "/User/path"
    os.makedirs(save_dir, exist_ok=True)

    all_results = []

    # Batch progress with tqdm
    for i in tqdm(range(0, len(df), batch_size), desc="All batches"):
        batch_data = df.iloc[i:i + batch_size]
        batch_id = i // batch_size + 1

        batch_results = process_batch_parallel(batch_data, batch_id, max_workers)
        all_results.extend(batch_results)

        # Interim save
        temp_data = df.iloc[:len(all_results)].copy()
        temp_data["LLM_Author_RegionCountry_raw"] = all_results
        temp_data["LLM_Author_Region"] = temp_data["LLM_Author_RegionCountry_raw"].apply(
            lambda x: parse_region_country(x)["Region"]
        )
        temp_data["LLM_Author_Country"] = temp_data["LLM_Author_RegionCountry_raw"].apply(
            lambda x: parse_region_country(x)["Country"]
        )

        interim_path = os.path.join(
            save_dir, f"LLM_author_region_country_interim_{len(all_results):07d}.csv"
        )
        temp_data.to_csv(interim_path, index=False)
        print(f"💾 Saved interim file: {interim_path}")

    print(f"\n🎉 Completed all records: {len(all_results):,}")

    # Write final results back to df
    df["LLM_Author_RegionCountry_raw"] = all_results
    df["LLM_Author_Region"] = df["LLM_Author_RegionCountry_raw"].apply(
        lambda x: parse_region_country(x)["Region"]
    )
    df["LLM_Author_Country"] = df["LLM_Author_RegionCountry_raw"].apply(
        lambda x: parse_region_country(x)["Country"]
    )

    # Final save
    final_path = os.path.join(
        save_dir, "author_region_country_LLM_classified_final.csv"
    )
    df.to_csv(final_path, index=False)
    print(f"\n💾 Saved final results: {final_path}")

    # Summary
    print("\n" + "=" * 60)
    print("📈 Summary")
    print("=" * 60)

    print("\n📋 First 10 results:")
    print(
        df[
            [
                "Author",
                "Year",
                "LLM_Author_RegionCountry_raw",
                "LLM_Author_Region",
                "LLM_Author_Country",
            ]
        ].head(10)
    )

    print("\n📊 Counts by Region:")
    print(df["LLM_Author_Region"].value_counts(dropna=False))

    print("\n📊 Top 20 Countries:")
    print(df["LLM_Author_Country"].value_counts(dropna=False).head(20))

    # Error count
    error_count = (
        df["LLM_Author_RegionCountry_raw"]
        .astype(str)
        .str.contains("Error", na=False)
        .sum()
    )
    total_count = len(df)
    print(f"\n❌ Errors: {error_count:,} ({error_count/total_count*100:.2f}%)")

    print("\n✅ Done!")
    return df


# ===================== ⑤ Quick preview (first row) =====================

def preview_first_one(df):
    """
    Classify only the first record with the LLM and print the result immediately.
    """
    print("\n===============================")
    print("🔍 Preview: First 1 LLM Result")
    print("===============================\n")

    if len(df) == 0:
        print("The dataset is empty.")
        return

    row = df.iloc[0]
    author = row["Author"]
    year = row["Year"]

    if pd.isna(author) or str(author).strip() == "":
        raw = "Skipped: Author missing"
    else:
        raw = classify_author_region_country(author, year)

    parsed = parse_region_country(raw)

    print(f"[0] Author: {author}")
    print(f"    Year:   {year}")
    print(f"    Raw:    {raw}")
    print(f"    → Region: {parsed['Region']}, Country: {parsed['Country']}\n")

    print("===============================================")
    print("Preview finished. Starting full processing with tqdm.")
    print("===============================================\n")


# ===================== ⑥ Runner wrapper =====================

def run_classification():
    """
    Runner wrapper:
    - Show an immediate preview for the first record
    - Then run full processing with tqdm
    """
    global df
    print("🚀 Starting LLM-based author region/country classification...")
    print(f"📊 Total records: {len(df):,}")
    print("📌 Rows with missing Author will be skipped (Region/Country remain Unknown).")
    print("📌 Rows with missing Year will be inferred using Author only.")

    # Quick preview for the first record
    preview_first_one(df)

    response = input("Start processing the full dataset? (y/n): ")
    if response.lower() != "y":
        print("Cancelled.")
        return

    result_data = main_processing()
    return result_data


print("\n🔧 Ready!")
print("Prepare df first, then call run_classification().")
print("Example:")
print("  df = pd.read_csv('your_input.csv')")
print("  run_classification()")


In [ ]:
# ===================== Execution =====================

if __name__ == "__main__":

    # Run classification
    result_df = run_classification()

    # Display results after completion (optional)
    print("\n=== Top 5 rows of LLM inference results ===")
    print(result_df.head(5))

    print("\n🎉 All processing completed successfully!")
